In [15]:
# aggregate_climate_by_adm3.py

import duckdb
import pandas as pd

# Connect to DuckDB database (with spatial extension)
con = duckdb.connect('db/india_model.ddb')
con.execute("INSTALL spatial; LOAD spatial;")

# Aggregate climate data by ADM3 region and date using correct column names
query = """
SELECT
    m.GID_3 as gid,
    m.NAME_3 as name,
    c.date,
    AVG(c.t2m) AS avg_temperature,
    SUM(c.tp) AS total_precipitation,
    AVG(c.swvl1) AS avg_soil_moisture,
    AVG(c.u10) AS avg_wind_u,
    AVG(c.v10) AS avg_wind_v,
    AVG(c.ssrd) AS avg_solar_radiation
FROM climate_data AS c
LEFT JOIN point_to_region_mapping AS m
    ON c.latitude = m.latitude AND c.longitude = m.longitude
WHERE m.GID_3 IS NOT NULL
GROUP BY m.GID_3, m.NAME_3, c.date
ORDER BY m.GID_3, c.date
"""

# Run the query and save the result
df = con.execute(query).fetchdf()
df['date'] = pd.to_datetime(df['date']).dt.date

# Convert temperature from Kelvin to Celsius
if 'avg_temperature' in df.columns:
    df['avg_temperature'] = df['avg_temperature'] - 273.15

# Add primary key and title columns
df['primary_key'] = df['gid'].astype(str) + '_' + df['date'].astype(str)
df['title'] = df['name'].astype(str) + ' - ' + df['date'].astype(str)


# Move primary_key to the first column and title to the second
cols = ['primary_key', 'title'] + [col for col in df.columns if col not in ['primary_key', 'title']]
df = df[cols]

df.to_parquet("output/climate_agg_by_adm3.parquet")

print("Aggregated climate data per ADM3 saved to output/climate_agg_by_adm3.parquet with primary_key and title columns added. Temperature is now in Celsius.")

Aggregated climate data per ADM3 saved to output/climate_agg_by_adm3.parquet with primary_key and title columns added. Temperature is now in Celsius.


# Foundry Time Series Sync Format for Climate Data

This notebook cell demonstrates how to transform aggregated climate data into the long-format structure required for Foundry time series syncs. Each row represents a single value for a region, parameter, and timestamp, making it easy to add new parameters and fully compatible with Foundry's time series tools.

In [16]:
# After df is created and temperature is converted to Celsius

# Melt the DataFrame to long format for time series sync
long_df = df.melt(
    id_vars=["gid", "name", "date", "primary_key", "title"],
    value_vars=[
        "avg_temperature",
        "total_precipitation",
        "avg_soil_moisture",
        "avg_wind_u",
        "avg_wind_v",
        "avg_solar_radiation"
    ],
    var_name="parameter",
    value_name="value"
)

# Rename columns to match Foundry time series sync requirements
long_df = long_df.rename(columns={
    "gid": "series_id",   # Series ID for the time series
    "date": "timestamp"   # Timestamp column
})

# Select and order columns for Foundry time series sync
long_df = long_df[[
    "series_id",    # Region GID (ADM3)
    "timestamp",    # Timestamp (date)
    "parameter",    # Parameter name (e.g., avg_temperature)
    "value",        # Value for the parameter
    "name",         # Region name (optional, for context)
    "primary_key",  # Unique row key (optional)
    "title"         # Title (optional)
]]

# Save to Parquet for Foundry time series sync
long_df.to_parquet("output/climate_agg_by_adm3_timeseries.parquet")

print("Saved long-format time series sync file for Foundry: output/climate_agg_by_adm3_timeseries.parquet")

Saved long-format time series sync file for Foundry: output/climate_agg_by_adm3_timeseries.parquet
